In [ ]:
!pip install transformers datasets accelerate evaluate bitsandbytes 

In [ ]:
!pip install git+https://github.com/huggingface/trl
!pip install git+https://github.com/huggingface/peft

from huggingface_hub import login
from google.colab import userdata

import os
HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

login(
  token=HUGGINGFACE_TOKEN, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [1]:
isWin=False
shard=10

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

maxInput=2000
IGNORE_INDEX = -100

In [2]:
from datasets import load_dataset
if isWin:
    dataset=load_dataset("json", 
        data_files={"train":"I:/kaggle/tokened_deepseek_math.stackexchange.com_shard" + str(shard) + "_train_dataset.json", 
                    "test":"I:/kaggle/tokened_deepseek_math.stackexchange.com_shard" + str(shard) + "_test_dataset.json"}, 
                     cache_dir="I:/kaggle/cache") # 是token,可以直接训练,不需要在token化
else:
    # dataset=load_dataset("json", 
        # data_files={"train":"/mnt/data/data/tokened_deepseek_math.stackexchange.com_shard" + str(shard) + "_train_dataset.json", 
        #             "test":"/mnt/data/data/tokened_deepseek_math.stackexchange.com_shard" + str(shard) + "_test_dataset.json"}, 
        #              cache_dir="I:/kaggle/cache") # 是token,可以直接训练,不需要在token化
    dataset=load_dataset("json", 
        data_files={"train":"/mnt/data/data/tokened_deepseek_math.OpenMathIns_train_dataset.json", 
                    "test":"/mnt/data/data/tokened_deepseek_math.OpenMathIns_test_dataset.json"}, 
                     cache_dir="I:/kaggle/cache") # 是token,可以直接训练,不需要在token化
dataset 

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 788416 examples [00:05, 134959.98 examples/s]
Generating test split: 1580 examples [00:00, 19298.03 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 788416
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1580
    })
})

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, BitsAndBytesConfig, PreTrainedModel, pipeline
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
if isWin:
    model_name = "I:/kaggle/deepseek-math-7b-it"
    tokenizer = AutoTokenizer.from_pretrained("I:/kaggle/deepseek-math-7b-it")
    attn_implementation=None
    bnb_config=None
else:
    model_name = "/mnt/data/deepseek-math-7b-it"
    # model_name = "/mnt/data/deepseek-math-7b-it-Train" + str(shard) + "Merged"
    tokenizer = AutoTokenizer.from_pretrained("/mnt/data/deepseek-math-7b-it")
    attn_implementation="flash_attention_2"

tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             attn_implementation = attn_implementation,
                                             quantization_config=bnb_config)

model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id
model, tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [00:35<00:00, 11.73s/it]


(LlamaForCausalLM(
   (model): LlamaModel(
     (embed_tokens): Embedding(102400, 4096)
     (layers): ModuleList(
       (0-29): 30 x LlamaDecoderLayer(
         (self_attn): LlamaFlashAttention2(
           (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
           (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
           (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
           (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
           (rotary_emb): LlamaRotaryEmbedding()
         )
         (mlp): LlamaMLP(
           (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
           (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
           (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
           (act_fn): SiLU()
         )
         (input_layernorm): LlamaRMSNorm()
         (post_attention_layernorm): LlamaRMSNorm()
      

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 788416
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1580
    })
})

In [6]:
dataset["train"].select([0]) # dataset分片成dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1
})

In [7]:
import gc
gc.collect()

33

## 4. Fine-tune LLM using `trl` and the `SFTTrainer`

use [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` to fine-tune our model. The `SFTTrainer` makes it straightfoward to supervise fine-tune open LLMs. The `SFTTrainer` is a subclass of the `Trainer` from the `transformers` library and supports all the same features, including logging, evaluation, and checkpointing, but adds additiional quality of life features, including:
* Dataset formatting, including conversational and instruction format
* Training on completions only, ignoring prompts
* Packing datasets for more efficient training
* PEFT (parameter-efficient fine-tuning) support including Q-LoRA
* Preparing the model and tokenizer for conversational fine-tuning (e.g. adding special tokens)

We will use the dataset formatting, packing and PEFT features in our example. As peft method we will use [QLoRA](https://arxiv.org/abs/2305.14314) a technique to reduce the memory footprint of large language models during finetuning, without sacrificing performance by using quantization. If you want to learn more about QLoRA and how it works, check out [Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA](https://huggingface.co/blog/4bit-transformers-bitsandbytes) blog post.

Next, we will load our LLM. For our use case we are going to use CodeLlama 7B. CodeLlama is a Llama model trained for general code synthesis and understanding.
But we can easily swap out the model for another model, e.g. [Mistral](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) or [Mixtral](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1) models, TII [Falcon](https://huggingface.co/tiiuae/falcon-40b), or any other LLMs by changing our `model_id` variable. We will use bitsandbytes to quantize our model to 4-bit.

_Note: Be aware the bigger the model the more memory it will require. In our example we will use the 7B version, which can be tuned on 24GB GPUs. If you have a smaller GPU._

Correctly, preparing the LLM and Tokenizer for training chat/conversational models is crucial. We need to add new special tokens to the tokenizer and model and teach to understand the different roles in a conversation. In `trl` we have a convinient method called [setup_chat_format](https://huggingface.co/docs/trl/main/en/sft_trainer#add-special-tokens-for-chat-format), which:
* Adds special tokens to the tokenizer, e.g. `<|im_start|>` and `<|im_end|>`, to indicate the start and end of a conversation.
* Resizes the model’s embedding layer to accommodate the new tokens.
* Sets the `chat_template` of the tokenizer, which is used to format the input data into a chat-like format. The default is `chatml` from OpenAI.

In [8]:
import gc
gc.collect()

0

In [9]:
tokenizer("Latex LaTex",
                return_tensors="pt",
                padding="longest",
                max_length=4096,
                truncation=True,
            )#.input_ids.ne(tokenizer.pad_token_id).sum().item()

{'input_ids': tensor([[100000,  56846,     87,   1689,  39422]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [10]:
import transformers
from typing import Optional, Dict, Sequence
from dataclasses import dataclass, field
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = [torch.tensor(x) for x in input_ids]
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=False, padding_value=self.tokenizer.pad_token_id
        )
        labels = [torch.tensor(x) for x in labels]
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=False, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
data_collator

DataCollatorForSupervisedDataset(tokenizer=LlamaTokenizerFast(name_or_path='/mnt/data/deepseek-math-7b-it', vocab_size=100000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	100000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	100001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
})

In [11]:
dataset["train"]=dataset["train"].shuffle()

In [12]:
type(dataset["train"])

datasets.arrow_dataset.Dataset

In [13]:
data_module = dict(train_dataset=dataset["train"], eval_dataset=dataset["test"], data_collator=data_collator)
data_module

{'train_dataset': Dataset({
     features: ['input_ids', 'labels'],
     num_rows: 788416
 }),
 'eval_dataset': Dataset({
     features: ['input_ids', 'labels'],
     num_rows: 1580
 }),
 'data_collator': DataCollatorForSupervisedDataset(tokenizer=LlamaTokenizerFast(name_or_path='/mnt/data/deepseek-math-7b-it', vocab_size=100000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	100000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	100001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 })}

In [14]:
dataset["test"][0]['input_ids'], dataset["test"][0]['labels'],

([100000,
  5726,
  25,
  1724,
  1313,
  4864,
  8095,
  1217,
  207,
  18,
  15,
  463,
  30,
  185,
  7900,
  2806,
  3458,
  457,
  3458,
  11,
  285,
  565,
  463,
  2328,
  3510,
  1957,
  254,
  2328,
  3510,
  2383,
  357,
  63962,
  40639,
  410,
  1957,
  254,
  3510,
  1166,
  655,
  549,
  3510,
  317,
  22734,
  207,
  185,
  185,
  77398,
  25],
 [100000,
  549,
  9966,
  45867,
  280,
  207,
  18,
  15,
  317,
  363,
  17,
  357,
  3560,
  207,
  18,
  357,
  3560,
  207,
  20,
  1332,
  185,
  6902,
  6088,
  280,
  207,
  18,
  15,
  1534,
  330,
  280,
  254,
  1020,
  363,
  17,
  61,
  64,
  357,
  3560,
  207,
  18,
  61,
  65,
  357,
  3560,
  207,
  20,
  61,
  66,
  3,
  1066,
  363,
  15,
  357,
  2848,
  245,
  357,
  2848,
  207,
  16,
  1348,
  363,
  15,
  357,
  2848,
  270,
  357,
  2848,
  207,
  16,
  1348,
  285,
  363,
  15,
  357,
  2848,
  258,
  357,
  2848,
  207,
  16,
  1332,
  185,
  11775,
  11,
  745,
  418,
  3309,
  16,
  10,
  16,
  4951,


In [15]:
tokenizer.decode(dataset["test"][0]['labels'], skip_special_tokens=True)

'The prime factorization of 30 is $2 \\cdot 3 \\cdot 5$.\nAny factor of 30 must be of the form $2^a \\cdot 3^b \\cdot 5^c$ where $0 \\leq a \\leq 1$, $0 \\leq b \\leq 1$, and $0 \\leq c \\leq 1$.\nThus, there are $(1+1)(1+1)(1+1) = \\boxed{8}$ positive factors of 30.\nThe answer is: 8'

In [16]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): 

In [17]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
import torch.nn.functional as F

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=128,
        bias="none",
        # target_modules=r"model\.layers\.(0|1|2)\..*\..*_proj", # all-linear "model.layers.30.mlp.down_proj" 2 5 8 11 14 17 20 23 26 29 
        # target_modules=r".*\..*\..*_proj", # 好的能用 (0|1|2|3|4|5|6|7|8|9) |1|2|3|4|5|6|7|8|9|10|11|12|13|14
        # target_modules=r"model.embed_tokens",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
        # modules_to_save=["model.embed_tokens"], 不能加,加了内存爆了
)

In [18]:
model.add_adapter(peft_config, "peft_adapter1")
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(102400, 4096)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (peft_adapter1): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (peft_adapter1): Linear(in_features=4096, out_features=128, bias=False)
            )
            (lora_B): ModuleDict(
              (peft_adapter1): Linear(in_features=128, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (peft_adapter1):

In [19]:
def print_trainable_parameters(model):
     trainable_params = 0
     all_param = 0
     for _, param in model.named_parameters():
         all_param += param.numel()
         if param.requires_grad:
             trainable_params += param.numel()
     print(
         f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
     )
print_trainable_parameters(model)

trainable params: 299827200 || all params: 4174565376 || trainable%: 7.18


In [20]:
# 打印总参数数量
total_params = sum(p.numel() for p in model.parameters())
print(f'Total parameters: {total_params}')

# 打印可训练参数数量
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable parameters: {trainable_params}')

# 打印不可训练参数数量
non_trainable_params = total_params - trainable_params
print(f'Non-trainable parameters: {non_trainable_params}')

Total parameters: 4174565376
Trainable parameters: 299827200
Non-trainable parameters: 3874738176


In [21]:
from transformers import TrainingArguments
import os
#32batch,1e-5,norm 0.03 不收敛
#32batch,1e-5,norm 0.1 不收敛
#32batch,1e-5,norm 1.0 不收敛
#32batch,1e-5,norm 5.0 

if isWin:
    output_dir = "I:/kaggle/deepseek-math-7b-it-Train" + str(shard)
    learning_rate=2e-5
    g_acc_steps=16
    warmup_ratio=0.01
    report_to="tensorboard"
else:
    output_dir = "/mnt/data/deepseek-math-7b-it-Train" + str(shard)
    learning_rate=1e-6 # when grad is small from 1e-7 to 2e-7 to 5e-7 to 1e-6 to 3e-6 to 1e-5 3e-5 5e-5
    g_acc_steps=128 # when grad is smal,from 32 to 128
    warmup_ratio=0.05
    report_to="none"
    
train_args = TrainingArguments(
    output_dir=output_dir, # directory to save and repository id
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=2,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
gradient_accumulation_steps=g_acc_steps,      # number of steps before performing a backward/update pass
    gradient_checkpointing=True,             # use gradient checkpointing to save memory
    optim="adamw_torch_fused",                # use fused adamw optimizer
    logging_steps=1,                          # log every 10 steps
    save_strategy="steps",                    # save checkpoint every   epoch,steps,no
    save_steps = 50,
    learning_rate=learning_rate,              # learning rate, based on QLoRA paper 1e-4收敛慢,用3e-4
    bf16=False,                              # use bfloat16 precision if you have supported GPU
    tf32=False,                              # use tf32 precision if you have supported GPU
    max_grad_norm=1,                       # max gradient norm based on QLoRA paper
    warmup_ratio=warmup_ratio,                # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant_with_warmup", # use constant learning rate scheduler
    push_to_hub=False,                       # push model to hub
    report_to=report_to,                      # report metrics to tensorboard
    fp16 = False,                            # 必须不能使用混合精度训练,用了就nan
    evaluation_strategy = "steps",            # steps / no / epoch
eval_steps=len(dataset["train"])//g_acc_steps//7,
    per_device_eval_batch_size=1,
)

if not os.path.exists(train_args.output_dir):
    os.makedirs(train_args.output_dir)

In [22]:
gc.collect(),torch.cuda.empty_cache()

(452, None)

In [23]:
param_dict = {}
param_stats = {}
import json

for name, param in model.named_parameters():
    param_dict[name] = {'requires_grad': param.requires_grad}
    if param.dtype not in [torch.uint8]:
        param_stats[name] = {
        'mean': torch.mean(param).item(),
        'max': torch.max(param).item(),
        'min': torch.min(param).item(),
        'median': torch.median(param).item(),
        'variance': torch.var(param).item(),
#         '25th percentile': torch.quantile(param.to(torch.float32), 0.25),
#         '75th percentile': torch.quantile(param.to(torch.float32), 0.75)
        }
        print(f"Name: {name}", f"Shape: {param.shape}", f"Type: {type(param)}", param_stats[name])
    else:
        print(f"Name: {name}", f"Shape: {param.shape}", f"Type: {type(param)}")
    
json_str = json.dumps(param_dict)
with open(train_args.output_dir + '/model_parameters.json', 'w') as f:
    f.write(json_str)

print("参数可导信息已保存到 "+ train_args.output_dir + "/model_parameters.json 文件中。")

Name: model.embed_tokens.weight Shape: torch.Size([102400, 4096]) Type: <class 'torch.nn.parameter.Parameter'> {'mean': -0.00049591064453125, 'max': 4.5, 'min': -4.65625, 'median': -0.00029754638671875, 'variance': 0.03173828125}
Name: model.layers.0.self_attn.q_proj.base_layer.weight Shape: torch.Size([8388608, 1]) Type: <class 'bitsandbytes.nn.modules.Params4bit'>
Name: model.layers.0.self_attn.q_proj.lora_A.peft_adapter1.weight Shape: torch.Size([128, 4096]) Type: <class 'torch.nn.parameter.Parameter'> {'mean': 8.001355126907583e-06, 'max': 0.015624992549419403, 'min': -0.015624985098838806, 'median': 3.546103835105896e-05, 'variance': 8.150124631356448e-05}
Name: model.layers.0.self_attn.q_proj.lora_B.peft_adapter1.weight Shape: torch.Size([4096, 128]) Type: <class 'torch.nn.parameter.Parameter'> {'mean': 0.0, 'max': 0.0, 'min': 0.0, 'median': 0.0, 'variance': 0.0}
Name: model.layers.0.self_attn.k_proj.base_layer.weight Shape: torch.Size([8388608, 1]) Type: <class 'bitsandbytes.nn.

In [24]:
for name, layer in model.named_modules():
    if hasattr(layer, 'weight'):
        print(f"Layer Name: {name}", f",Weight Shape: {layer.weight.shape}", f",Weight requires_grad: {layer.weight.requires_grad}")

Layer Name: model.embed_tokens ,Weight Shape: torch.Size([102400, 4096]) ,Weight requires_grad: False
Layer Name: model.layers.0.self_attn.q_proj ,Weight Shape: torch.Size([8388608, 1]) ,Weight requires_grad: False
Layer Name: model.layers.0.self_attn.q_proj.base_layer ,Weight Shape: torch.Size([8388608, 1]) ,Weight requires_grad: False
Layer Name: model.layers.0.self_attn.q_proj.lora_A.peft_adapter1 ,Weight Shape: torch.Size([128, 4096]) ,Weight requires_grad: True
Layer Name: model.layers.0.self_attn.q_proj.lora_B.peft_adapter1 ,Weight Shape: torch.Size([4096, 128]) ,Weight requires_grad: True
Layer Name: model.layers.0.self_attn.k_proj ,Weight Shape: torch.Size([8388608, 1]) ,Weight requires_grad: False
Layer Name: model.layers.0.self_attn.k_proj.base_layer ,Weight Shape: torch.Size([8388608, 1]) ,Weight requires_grad: False
Layer Name: model.layers.0.self_attn.k_proj.lora_A.peft_adapter1 ,Weight Shape: torch.Size([128, 4096]) ,Weight requires_grad: True
Layer Name: model.layers.0.s

In [25]:
from transformers import Trainer, TrainingArguments
import torch
import torch.nn.functional as F
from torch.optim import AdamW

input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.randint(5, (3,), dtype=torch.int64)
print(target)
loss = F.cross_entropy(input, target)
print(loss)
loss.backward(), target

tensor([[ 0.5653,  0.0944,  1.0561, -0.5761, -2.2156],
        [-0.7554,  0.4029,  2.0111,  0.5498, -0.0501],
        [-1.3394, -0.7353, -1.7051,  0.4580,  0.2896]], requires_grad=True)
tensor([1, 3, 0])
tensor(2.1309, grad_fn=<NllLossBackward0>)


(None, tensor([1, 3, 0]))

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

# logits 是模型的输出，形状为 (max_seq_length, batch_size, num_classes)
# targets 是真实的标签序列，形状为 (batch_size, target_length)
# input_lengths 是每个样本在 logits 中的有效序列长度
# target_lengths 是每个样本在 targets 中的有效序列长度
batch_size = 1
max_seq_length = 10
num_classes = 5
target_length = 8

logits = torch.randn(max_seq_length, batch_size, num_classes, requires_grad=True)
print("logits", logits)
targets = torch.randint(0, num_classes, (batch_size, target_length), dtype=torch.long)
print("targets", targets)

input_lengths = (max_seq_length, )
target_lengths = (target_length, )
# 创建 CTCLoss 实例
ctc_loss = nn.CTCLoss(zero_infinity=True)
print("logits.log_softmax(2)", logits.softmax(2))
# 计算损失
loss = ctc_loss(logits.log_softmax(2), targets, input_lengths, target_lengths,)

# 打印损失
print(f"CTC Loss: {loss.item()}")
# 反向传播
# optimizer = optim.Adam([logits])
# optimizer.zero_grad()
# loss.backward()
# # 更新权重
# optimizer.step()

logits tensor([[[ 0.2853, -0.2949,  0.7317, -0.1566,  0.1362]],

        [[-0.9048, -1.2314,  1.2278,  1.1932,  0.3981]],

        [[ 0.2849,  0.2155,  0.2127,  0.0457,  0.4772]],

        [[ 1.7091, -0.8062, -0.2040, -1.2841,  0.7576]],

        [[ 0.6326,  1.2340,  0.5028,  0.0084, -0.5258]],

        [[-0.7823, -1.1374,  0.7112, -0.5984, -0.9900]],

        [[-0.8388, -1.0517,  1.6804,  0.5339, -0.3498]],

        [[-0.8874,  0.2125, -1.6582, -0.4181, -0.7699]],

        [[ 0.4796, -2.3274,  0.9537, -0.2509, -0.5704]],

        [[-1.5211, -1.5746,  0.3632,  2.0650, -1.1210]]], requires_grad=True)
targets tensor([[1, 3, 4, 1, 0, 4, 4, 3]])
logits.log_softmax(2) tensor([[[0.2161, 0.1210, 0.3377, 0.1389, 0.1862]],

        [[0.0455, 0.0328, 0.3837, 0.3707, 0.1674]],

        [[0.2057, 0.1919, 0.1913, 0.1619, 0.2493]],

        [[0.6007, 0.0486, 0.0887, 0.0301, 0.2320]],

        [[0.2197, 0.4008, 0.1929, 0.1177, 0.0690]],

        [[0.1224, 0.0858, 0.5451, 0.1471, 0.0995]],

        [[

In [30]:
import json
from transformers import TrainerCallback

class SaveTrainingRecordCallback(TrainerCallback):
    inputs = {}
    
    def on_step_end(self, args, state, control, **kwargs): 
        # print("on_step_end state", state)
        epoch = state.epoch
        global_step = state.global_step
        
        if len(state.log_history) > 0 and state.log_history[-1].get("loss") is not None:
            loss = state.log_history[-1]["loss"]
            grad_norm = state.log_history[-1]["grad_norm"]
            learning_rate = state.log_history[-1]["learning_rate"]

            if epoch > 0. and (loss > -1000.  or grad_norm > 0.01):#epoch>0.5 and (loss>-0.95 or grad_norm>0.15) | epoch>1. and (loss>-0.95 or grad_norm>0.09)
                with open('largeLossRecords.deepseekmath.MetaMathQA' + str(shard) + '.json', 'a') as f:
                    aim = {"input_ids" : SaveTrainingRecordCallback.inputs["input_ids"].flatten().tolist(), 
                           "labels" : SaveTrainingRecordCallback.inputs["labels"].flatten().tolist(), "learning_rate" : learning_rate, 
                           "global_step" : global_step, "loss" : loss, "grad_norm" : grad_norm, "epoch" : epoch}
                    json.dump(aim, f)
                    f.write('\n')

In [31]:
import torch
import torch.nn as nn
a=torch.rand(3,1,2)
b = a.unsqueeze(-1)
a,a.shape,len(a),b.shape,len(b)

(tensor([[[0.6073, 0.8622]],
 
         [[0.2123, 0.2941]],
 
         [[0.3173, 0.2973]]]),
 torch.Size([3, 1, 2]),
 3,
 torch.Size([3, 1, 2, 1]),
 3)

In [32]:
import torch.nn.functional as F
F.softmax(torch.tensor([0.,0.,0.,1]))

/tmp/ipykernel_6700/1624711620.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.tensor([0.,0.,0.,1]))


tensor([0.1749, 0.1749, 0.1749, 0.4754])

In [33]:
from transformers import Trainer
import torch.nn.functional as F
from torch.nn.functional import cross_entropy

class CustomTrainer(Trainer):
    counter = 0
        
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.
        Subclass and override for custom behavior.
        """
        SaveTrainingRecordCallback.inputs = inputs
        
        if "labels" in inputs:
            labels = inputs["labels"]
        else:
            labels = None
        outputs = model(**inputs)
        # print("in compute_loss, labels.shape",labels.shape, "outputs['logits'].shape", outputs['logits'].shape)
        # Save past state if it exists   # TODO: this needs to be fixed and made cleaner later.
        # if self.args.past_index >= 0:
        #     self._past = outputs[self.args.past_index]

        if labels is not None and outputs["logits"] is not None and len(outputs["logits"]) > 0 and len(labels) > 0:
            # loss = label_smoother(outputs, labels, shift_labels=False)
            # loss = self.blend_loss( outputs["logits"], labels, len(inputs["labels"]), doPrint=True )
            # print("outputs['logits'].shape", outputs['logits'].shape, "labels.shape", labels.shape, "outputs['logits'].softmax(2)", outputs["logits"].softmax(2))
            loss = ctc_loss(outputs["logits"].log_softmax(2), labels.view(1,-1), (len(inputs["input_ids"]),), (len(inputs["labels"]),))
            new_first_dim = max(len(inputs["input_ids"]), len(inputs["labels"]))
        else:
            if isinstance(outputs, dict) and "loss" not in outputs:
                return 0.
                # raise ValueError(
                #     "The model did not return a loss from the inputs, only the following keys: "
                #     f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
                # )
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        if CustomTrainer.counter % train_args.gradient_accumulation_steps == 0:
            grad_norms = [torch.norm(grad) for grad in model.parameters() if grad is not None and grad.dtype not in [torch.uint8] and grad.requires_grad]
            mean_grad_norm = torch.mean(torch.stack(grad_norms))
            loss=loss + mean_grad_norm / 10000.
            print("custom_loss_function steps:", CustomTrainer.counter, "output dim:", new_first_dim, "loss:", loss.item(), "Mean Grad Norm:", mean_grad_norm.item(),)
            
        CustomTrainer.counter = CustomTrainer.counter + 1
        
        return (loss, outputs) if return_outputs else loss

    def blend_loss(self, logits, labels, alength, doPrint=True):
        # 预测的长度 > 问题的长度    训练数据labels的长度一定大于问题的长度
        if len(logits) > alength:
            loss = self.cosine_loss(logits, labels, doPrint=False) # logits[-alength:]
            loss2 = 0. # self.cross_loss(logits[:alength], labels[:alength], doPrint=False)
        elif len(logits) == alength: # 预测的长度 == 问题的长度
            loss = self.cosine_loss(logits, labels, doPrint=False)
            loss2 = 0.
        else:
            loss = self.cosine_loss(logits, labels, doPrint=False)
            loss2 = 0.

        result = 100000. * loss # -10. / (torch.sinh(loss - 1.1) ) - 7.487 # result 最小为 -5.001,其斜率为 2.562

        mean_grad_norm = 0.
        if doPrint:
            if CustomTrainer.counter % train_args.gradient_accumulation_steps == 0:
                grad_norms = [torch.norm(grad) for grad in model.parameters() if grad is not None and grad.dtype not in [torch.uint8] and grad.requires_grad]
                mean_grad_norm = torch.mean(torch.stack(grad_norms))
                print("custom_loss_function steps:", CustomTrainer.counter, "output dim:", 4096, "loss:", result.item(), "Mean Grad Norm:", mean_grad_norm.item(),)

            CustomTrainer.counter = CustomTrainer.counter + 1
            # return result + loss2 / 10. + mean_grad_norm / 10000.

        return result

    def cosine_loss(self, logits, labels, doPrint=True):
        # print("logits.shape", logits.shape)
        # print("labels.shape", labels.shape)

        logits_squeezed = logits.squeeze(1)  # logits的形状现在是[316, 32000]
        # # 归一化 光归一化不好，求和可能大于1很多
        # min_val = torch.min(logits_squeezed)
        # max_val = torch.max(logits_squeezed)
        # logits_squeezed = (logits_squeezed - min_val) / (max_val - min_val) 
        logits_squeezed = F.softmax(logits_squeezed, dim=1) # 这时候 logits_squeezed 才代表是每个标签或分类的概率
        # print("logits_squeezed.shape", logits_squeezed.shape)
        # print("torch.sum(logits_squeezed,1)", torch.sum(logits_squeezed,1) )
        # model.embed_tokens.weight Shape: torch.Size([32000, 4096])
        qz = model.get_parameter("model.embed_tokens.weight")
        logits_squeezed = logits_squeezed.to(qz.dtype)
        logits_squeezed = torch.matmul(logits_squeezed, qz) # logits的形状现在是[316, 4096]
        labels_squeezed = labels.squeeze(1)  # labels的形状现在是[438]

        # 确定新的第一个维度大小  填充的方法内存爆了 .最多填充到100, 再长内存爆了
        minToken = 4096 # 150很大，已经不能收敛，不能再调大。调大连loss都收敛不了。往小调整100太小。
        new_first_dim = minToken # min(minToken, max(logits_squeezed.shape[0], labels_squeezed.shape[0]) )

        embeddings2 = model.get_submodule("model.embed_tokens").forward(labels_squeezed) # [438, 4096]

        # 计算句子向量（这里取所有词向量的平均值）
        sentence_vector1 = logits_squeezed.mean(dim=0) # shape 4096
        sentence_vector2 = embeddings2.mean(dim=0) # shape 4096

        logits_squeezed = sentence_vector1.to(torch.bfloat16)
        labels_squeezed = sentence_vector2.to(torch.bfloat16)

        padded_logits = logits_squeezed
        padded_labels = labels_squeezed
        # 计算余弦相似度
        # loss = -torch.dot(padded_logits, padded_labels) / (torch.norm(padded_logits) * torch.norm(padded_labels))
        loss = torch.mean( (padded_logits - padded_labels)**2 )

        if doPrint:
            if CustomTrainer.counter % train_args.gradient_accumulation_steps == 0:
                grad_norms = [torch.norm(grad) for grad in model.parameters() if grad is not None and grad.dtype not in [torch.uint8] and grad.requires_grad]
                mean_grad_norm = torch.mean(torch.stack(grad_norms))
                print("custom_loss_function steps:", CustomTrainer.counter, "output dim:", new_first_dim, "loss:", loss.item(), "Mean Grad Norm:", mean_grad_norm.item(),)

            CustomTrainer.counter = CustomTrainer.counter + 1

        return loss

In [34]:
from transformers import Trainer

model.is_parallelizable = True
model.model_parallel = True
# trainer = Trainer(model=model, tokenizer=tokenizer, args=args, **data_module) 必须使用自己的类才能自定义损失函数
trainer = CustomTrainer(model=model, tokenizer=tokenizer, args=train_args, **data_module,
                       callbacks=[SaveTrainingRecordCallback()])
model.config.use_cache = False
trainer,gc.collect()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


(<__main__.CustomTrainer at 0x7fd650356410>, 40)

In [35]:
model.train()
model.training,gc.collect()

(True, 0)

In [36]:
model.active_adapters()

['peft_adapter1']

In [ ]:
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
trainer.train()
# trainer.train(resume_from_checkpoint="/mnt/data/deepseekTrain0/checkpoint-1000")

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


custom_loss_function steps: 0 output dim: 151 loss: 0.0003265986160840839 Mean Grad Norm: 3.265986204147339


Step,Training Loss,Validation Loss
879,0.680300,0.646730
1758,0.460700,0.554162


custom_loss_function steps: 128 output dim: 173 loss: 0.0003265986160840839 Mean Grad Norm: 3.265986204147339
custom_loss_function steps: 256 output dim: 497 loss: 50.42435073852539 Mean Grad Norm: 3.265986680984497
custom_loss_function steps: 384 output dim: 112 loss: 0.0003265987033955753 Mean Grad Norm: 3.2659871578216553
custom_loss_function steps: 512 output dim: 239 loss: 0.0003265988198108971 Mean Grad Norm: 3.265988349914551
custom_loss_function steps: 640 output dim: 256 loss: 0.0003265990235377103 Mean Grad Norm: 3.2659902572631836
custom_loss_function steps: 768 output dim: 308 loss: 0.00032659919816069305 Mean Grad Norm: 3.2659921646118164
custom_loss_function steps: 896 output dim: 168 loss: 0.00032659946009516716 Mean Grad Norm: 3.2659947872161865
custom_loss_function steps: 1024 output dim: 186 loss: 0.00032659975113347173 Mean Grad Norm: 3.2659976482391357
custom_loss_function steps: 1152 output dim: 98 loss: 0.0003266001003794372 Mean Grad Norm: 3.266000986099243
custo

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:192: UserWarning: Could not find a config file in /mnt/data/deepseek-math-7b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

custom_loss_function steps: 252800 output dim: 160 loss: 0.0003430109063629061 Mean Grad Norm: 3.4301092624664307
custom_loss_function steps: 252928 output dim: 72 loss: 0.0003430154756642878 Mean Grad Norm: 3.430154800415039
custom_loss_function steps: 253056 output dim: 511 loss: 0.00034301987034268677 Mean Grad Norm: 3.430198907852173
custom_loss_function steps: 253184 output dim: 84 loss: 5.550096035003662 Mean Grad Norm: 3.430238723754883
custom_loss_function steps: 253312 output dim: 103 loss: 0.0003430275828577578 Mean Grad Norm: 3.4302759170532227
custom_loss_function steps: 253440 output dim: 148 loss: 0.0003430308133829385 Mean Grad Norm: 3.4303083419799805
custom_loss_function steps: 253568 output dim: 363 loss: 0.0003430345095694065 Mean Grad Norm: 3.430345296859741
custom_loss_function steps: 253696 output dim: 399 loss: 0.00034303785650990903 Mean Grad Norm: 3.4303786754608154
custom_loss_function steps: 253824 output dim: 110 loss: 0.0003430410288274288 Mean Grad Norm: 3

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:192: UserWarning: Could not find a config file in /mnt/data/deepseek-math-7b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


custom_loss_function steps: 259200 output dim: 172 loss: 0.0003432258381508291 Mean Grad Norm: 3.4322586059570312
custom_loss_function steps: 259328 output dim: 262 loss: 0.00034322997089475393 Mean Grad Norm: 3.432299852371216
custom_loss_function steps: 259456 output dim: 106 loss: 0.0003432343655731529 Mean Grad Norm: 3.4323437213897705
custom_loss_function steps: 259584 output dim: 120 loss: 0.00034323870204389095 Mean Grad Norm: 3.432387113571167
custom_loss_function steps: 259712 output dim: 350 loss: 0.00034324266016483307 Mean Grad Norm: 3.432426691055298
custom_loss_function steps: 259840 output dim: 75 loss: 0.0003432461526244879 Mean Grad Norm: 3.4324615001678467
custom_loss_function steps: 259968 output dim: 52 loss: 5.000761032104492 Mean Grad Norm: 3.4325013160705566
custom_loss_function steps: 260096 output dim: 127 loss: 0.00034325409797020257 Mean Grad Norm: 3.4325411319732666
custom_loss_function steps: 260224 output dim: 201 loss: 0.00034325869637541473 Mean Grad Nor

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:192: UserWarning: Could not find a config file in /mnt/data/deepseek-math-7b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


custom_loss_function steps: 265600 output dim: 85 loss: 5.342731475830078 Mean Grad Norm: 3.434378147125244
custom_loss_function steps: 265728 output dim: 156 loss: 0.00034344065352343023 Mean Grad Norm: 3.4344067573547363
custom_loss_function steps: 265856 output dim: 357 loss: 0.00034344420419074595 Mean Grad Norm: 3.4344420433044434
custom_loss_function steps: 266112 output dim: 136 loss: 0.0003434519749134779 Mean Grad Norm: 3.4345197677612305
custom_loss_function steps: 266240 output dim: 124 loss: 0.00034345564199611545 Mean Grad Norm: 3.434556484222412
custom_loss_function steps: 266368 output dim: 233 loss: 0.0003434587852098048 Mean Grad Norm: 3.4345879554748535
custom_loss_function steps: 266496 output dim: 95 loss: 0.00034346242318861187 Mean Grad Norm: 3.434624433517456
custom_loss_function steps: 266624 output dim: 173 loss: 0.0003434657119214535 Mean Grad Norm: 3.434657335281372
custom_loss_function steps: 266752 output dim: 58 loss: 5.861666202545166 Mean Grad Norm: 3.43

In [ ]:
gc.collect()
trainer.save_state()

In [ ]:
trainer.save_model() # 用它保存,用 AutoPeftModelForCausalLM 读取

Start training our model by calling the `train()` method on our `Trainer` instance. This will start the training loop and train our model for 3 epochs. Since we are using a PEFT method, we will only save the adapted model weights and not the full model.

It took ~3.5 hrs on the free T4 GPU in Google Colab to finish training.

In [ ]:
# free the memory again
# del model
# del trainer
torch.cuda.empty_cache()

In [ ]:
# MERGE PEFT AND BASE MODEL 官方参数不要变 , 不要用量化加载合并, 合并的模型精度不行 报废
import torch
from peft import AutoPeftModelForCausalLM
checkPoint = "/checkpoint-1000"
checkPoint = ""
try:
    # Load PEFT model on CPU
    model = AutoPeftModelForCausalLM.from_pretrained(
        train_args.output_dir + checkPoint,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )
except Exception as e:
    print(e)
    model = AutoPeftModelForCausalLM.from_pretrained(
        train_args.output_dir + checkPoint,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )

In [ ]:
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
print("merge_and_unload finish")

In [ ]:
merged_model.save_pretrained(train_args.output_dir + "Merged", safe_serialization=True, max_shard_size="2GB")
print(train_args.output_dir + "Merged", "save_pretrained")

In [ ]:
gc.collect()

In [ ]:
def test_tokenize_function(examples, tokenizer):
    tokenizer.pad_token = tokenizer.eos_token
    
    question = examples["question"]
    context = ""
    system = "You're an expert Python programmer and mathematician. Help the user to solve this problem using code when necessary. \
Make sure to put the answer (and only answer) inside \\boxed{}."
    user = f"{question}\n\n{context}"
    generated_solution = ""
    Q = f"System:\n{system}\n\nUser:\n{user}Assistant:\n{generated_solution}"
    
    input_ids = tokenizer(Q, return_tensors="pt", padding="longest", max_length=4096,
                          truncation=True, add_special_tokens=True)

    return input_ids

In [ ]:
dataset["test"][0]

In [ ]:
input_ids = torch.tensor(dataset["test"][0]["input_ids"]).unsqueeze(0)
input_ids

In [ ]:
import time
del model, merged_model
for i in range(5):
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(1)

In [ ]:
if isWin:
    model_name = "I:/kaggle/deepseek-math-7b-it"
    tokenizer = AutoTokenizer.from_pretrained("I:/kaggle/deepseek-math-7b-it")
    attn_implementation=None
else:
    model_name = "/mnt/data/deepseek-math-7b-it"
    model_name = "/mnt/data/deepseek-math-7b-it-Train" + str(shard) + "Merged"
    tokenizer = AutoTokenizer.from_pretrained("/mnt/data/deepseek-math-7b-it")
    attn_implementation="flash_attention_2"

tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                             attn_implementation = attn_implementation,
                                             quantization_config=bnb_config)

model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id
model, tokenizer

In [ ]:
maxloutword=2000
max_new_tokens=None
outputs = model.generate(input_ids=input_ids, max_length=maxloutword, max_new_tokens=max_new_tokens, num_beams=1, pad_token_id=tokenizer.pad_token_id)
outputs

In [ ]:
resultTxt = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(resultTxt)

In [ ]:
label = tokenizer.decode(dataset["test"][0]["labels"], skip_special_tokens=True)
print(label)

In [ ]:
import shutil
src_file = 'largeLossRecords.deepseekmath.' + str(shard) + '.json'
dst_file = '/mnt/data/' + 'largeLossRecords.deepseekmath.' + str(shard) + '.json'
shutil.copy2(src_file, dst_file)

In [ ]:
shutil.copy2(train_args.output_dir + "/trainer_state.json", train_args.output_dir + "Merged/trainer_state.json")

In [ ]:
cmd = "cp " + train_args.output_dir + "/trainer_state.json " + train_args.output_dir + "Merged/trainer_state.json"
cmd

In [ ]:
!bash -c "$cmd"
!sleep 5s
!kill -SIGTERM 1

Let’s load our test dataset try to generate an instruction.